In [1]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 13.0 MB/s 
     |████████████████████████████████| 3.3 MB 39.1 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 895 kB 38.0 MB/s 
     |████████████████████████████████| 596 kB 41.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import torch
from transformers import BertForSequenceClassification
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import f1_score
import random

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
df = pd.read_csv("/content/drive/MyDrive/sentiment/data/train.csv")
df.head()

,content,sentiment
0,Не подошло.,negative
1,"товара нет, деньги вернул",negative
2,"[옵션]문풍지종류선택:3M실내용,규격선택:4. 대형",neutral
3,Kirain Alice rokoknya esse change wkwk,neutral
4,El hotel ideal para descansar y relajarte. El ...,positive


In [5]:
df.sentiment.value_counts()

neutral       8823
positive      8318
negative      7858
unassigned       1
Name: sentiment, dtype: int64

In [6]:
df = df[df.sentiment.isin(['neutral','positive','negative'])]

In [7]:
df.sentiment.value_counts()

neutral     8823
positive    8318
negative    7858
Name: sentiment, dtype: int64

In [8]:
possible_labels = df.sentiment.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [9]:
label_dict

{'negative': 0, 'neutral': 1, 'positive': 2}

In [10]:
df.sentiment = df['sentiment'].map(label_dict)
df.head()

,content,sentiment
0,Не подошло.,0
1,"товара нет, деньги вернул",0
2,"[옵션]문풍지종류선택:3M실내용,규격선택:4. 대형",1
3,Kirain Alice rokoknya esse change wkwk,1
4,El hotel ideal para descansar y relajarte. El ...,2


#### Training and Validation split

In [11]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.sentiment.values, 
                                                  test_size=0.2, 
                                                  random_state=42,
                                                  stratify=df.sentiment.values)

In [12]:
df['data_type'] = ['not_set']*df.shape[0]

In [13]:
df.head()

,content,sentiment,data_type
0,Не подошло.,0,not_set
1,"товара нет, деньги вернул",0,not_set
2,"[옵션]문풍지종류선택:3M실내용,규격선택:4. 대형",1,not_set
3,Kirain Alice rokoknya esse change wkwk,1,not_set
4,El hotel ideal para descansar y relajarte. El ...,2,not_set


In [14]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [15]:
df.groupby(['sentiment', 'data_type']).count()

content
sentiment data_type         
0         train         6287
          val           1571
1         train         7058
          val           1765
2         train         6654
          val           1664

### Loading Tokenizer and Encoding our Data

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [17]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].content.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt')

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].content.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt')

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].sentiment.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].sentiment.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [18]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [19]:
len(dataset_train)

19999

In [20]:
dataset_val.tensors

(tensor([[  101, 10224, 19157,  ...,     0,     0,     0],
         [  101,   146, 68312,  ...,     0,     0,     0],
         [  101, 11161, 33567,  ...,     0,     0,     0],
         ...,
         [  101, 11589, 30395,  ...,     0,     0,     0],
         [  101, 16060, 10519,  ...,     0,     0,     0],
         [  101, 31126, 91845,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([2, 0, 0,  ..., 2, 2, 1]))

###  Setting up BERT Pretrained Model

In [21]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-multilingual-cased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

### Creating Data Loaders

In [22]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32)

### Setting Up Optimizer and Scheduler

In [23]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [24]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

### Defining our Performance Metrics

In [25]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [26]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

###  Creating our Training Loop

In [27]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [29]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [30]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    torch.save(model.state_dict(), '/content/drive/MyDrive/sentiment/model/cased.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/5000 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.6048645402124617


  0%|          | 0/157 [00:00<?, ?it/s]

Validation loss: 0.5995868355700165
F1 Score (weighted): 0.8180339328605261


...

Validation loss: 0.7099668088185179
F1 Score (weighted): 0.8339110818758015


Epoch 3:   0%|          | 0/5000 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.32462099641505626


  0%|          | 0/157 [00:00<?, ?it/s]

Validation loss: 0.818988884472923
F1 Score (weighted): 0.8341275527011672


In [34]:
model.load_state_dict(torch.load('/content/drive/MyDrive/sentiment/model/cased.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [35]:
_,predictions, true_vals = evaluate(dataloader_val)

  0%|          | 0/157 [00:00<?, ?it/s]

In [36]:
accuracy_per_class(predictions, true_vals)

Class: negative
Accuracy:1243/1571

Class: neutral
Accuracy:1629/1765

Class: positive
Accuracy:1301/1664

